# Monte Carlo Scanning
## Importing Libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import henon_map as hm

from parameters import *

## Computing

In [2]:
problem = {
    'num_vars': 3,
    'names': ['alpha', 'theta1', 'theta2'],
    'bounds': [[-1, 1],
               [0, np.pi * 2],
               [0, np.pi * 2]]
    }
param_values = saltelli.sample(problem, mc_max_samples // 8 + 8)

alpha = np.array([np.arccos(p[0])/2 for p in param_values])
theta1 = np.array([p[1] for p in param_values])
theta2 = np.array([p[2] for p in param_values])

DA_3 = {}
error_3 = {}
data_3 = {}
for epsilon in tqdm(epsilons, desc="Monte Carlo"):
    # Data generation
    henon_engine = hm.radial_scan.generate_instance(d_r, alpha, theta1, theta2, epsilon)
    radiuses = henon_engine.compute(turn_sampling)
    
    data_3[epsilon] = radiuses

    # Computing DA
    
    for sample in mc_samples:
        average = np.average(np.power(radiuses[:sample], 4), axis=0)
        error = np.std(np.power(radiuses[:sample], 4), axis=0) / np.sqrt(sample)
        DA_3[(epsilon, sample)] = np.power(average, 1/4)
        error_3[(epsilon, sample)] = 0.25 * np.power(average, -3/4) * error
    

Monte Carlo: 100%|██████████| 1/1 [00:55<00:00, 55.38s/it]


### Saving data

In [3]:
with open("data/DA_3.pkl", 'wb') as f:
    pickle.dump(DA_3, f, protocol=4)
    
with open("data/error_3.pkl", 'wb') as f:
    pickle.dump(error_3, f, protocol=4)